## ИССЛЕДОВАНИЕ, СРАВНЕНИЕ И ВЫБОР АЛГОРИТМОВ КОМПЬЮТЕРНОГО ЗРЕНИЯ.

В ходе работы была проведена всесторонняя оценка современных методов компьютерного зрения для решения задачи идентификации особей крупных морских млекопитающих. Процесс выбора оптимального алгоритма включал как теоретический анализ существующих подходов, так и практическое тестирование на реальных данных. Основной задачей стало выявление алгоритма, который обеспечивал бы не только высокую точность распознавания, но и быструю обработку данных в условиях ограниченных вычислительных мощностей и разнообразных внешних факторов, таких как изменения погодных условий и динамическое окружение при съемке с дронов.
### 6.1 Цели исследования:
Оценить алгоритмы с точки зрения точности классификации.
Изучить вычислительную эффективность и устойчивость алгоритмов при работе с ограниченными ресурсами (например, при использовании на мобильных устройствах и дронов).
Проанализировать адаптивность алгоритмов к различным внешним условиям съемки, включая изменение освещенности, погодные условия и скорость движения объектов.

### 6.2 Алгоритмы для тестирования
`ResNet-50` — сеть обеспечивающая хорошую производительность даже при больших глубинах.
`EfficientNet-B0` — современная и оптимизированная архитектура, которая показывает хорошее соотношение между точностью и вычислительными затратами.
`MobileNetV2` — легковесная модель, хорошо подходящая для мобильных устройств и дронов.
6.3 Метрики для оценки
Точность (Top-1, %): процент правильно классифицированных объектов среди всех предсказаний.
Количество параметров (М): количество обучаемых параметров в модели, измеряемое в миллионах.
FLOPS (G): количество операций с плавающей точкой, необходимых для выполнения одного прохода модели, измеряемое в гигафлопсах.
Время обработки на GPU (мс): среднее время, необходимое для обработки одного изображения с использованием графического процессора.
Время обработки на CPU (мс): среднее время, необходимое для обработки одного изображения с использованием центрального процессора.


In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models
import torch
import time
from fvcore.nn import FlopCountAnalysis, parameter_count

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используется устройство: {device}")

Используется устройство: cuda


In [2]:
# Параметры
batch_size = 32
num_workers = 1
image_size = 224

# Трансформации данных
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


## Скачиваем датасет для тестирования на реальных данных

In [10]:
# Кастомный датасет
class CustomDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.data.iloc[idx, 1]  # Используем species как метку

        if self.transform:
            image = self.transform(image)

        label_idx = hash(label) % 1000
        return image, label_idx

# Загрузка датасета

# Ввиду ограничений и объема данных возможных загрузить на Github - необходимо использовать данные с https://www.kaggle.com/code/tarassssov/whales-users/
# и положить их в указанные папки
data_dir = '../../data/datasets'
csv_file = os.path.join(data_dir, 'data.csv')
img_dir = data_dir

if not os.path.exists(csv_file):
    raise FileNotFoundError(f"Файл {csv_file} не найден")

val_dataset = CustomDataset(csv_file, img_dir, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
val_loader

# Функции для оценки модели

In [4]:

def evaluate_model(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels).item()
            total += labels.size(0)
    accuracy = 100 * correct / total
    return accuracy

def measure_time(model, dataloader, device, num_batches=100):
    model.eval()
    timings = []
    with torch.no_grad():
        for i, (images, _) in enumerate(dataloader):
            if i >= num_batches:
                break
            images = images.to(device)
            start_time = time.time()
            outputs = model(images)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            end_time = time.time()
            timings.append((end_time - start_time) * 1000)  # в миллисекундах
    avg_time = sum(timings) / len(timings)
    return avg_time

In [ ]:
models_to_test = {
    'EfficientNet-B0': models.efficientnet_b0(pretrained=True).to(device),
    'ResNet-50': models.resnet50(pretrained=True).to(device),
    'MobileNetV2': models.mobilenet_v2(pretrained=True).to(device),
}

results = []

for model_name, model in models_to_test.items():
    print(f"\nТестируем модель: {model_name}")
    try:
        # Точность
        accuracy = evaluate_model(model, val_loader, device)

        # Количество параметров
        num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        num_params_m = num_params / 1e6  # в миллионах

        # FLOPS
        sample_images, _ = next(iter(val_loader))
        sample_images = sample_images.to(device)
        flops = FlopCountAnalysis(model, sample_images)
        flops_count = flops.total() / 1e9  # в гигафлопсах

        # Время обработки на GPU и CPU
        if device.type == 'cuda':
            gpu_time = measure_time(model, val_loader, device)
            model_cpu = model.to('cpu')
            cpu_time = measure_time(model_cpu, val_loader, torch.device('cpu'))
            model.to(device)
        else:
            gpu_time = None
            cpu_time = measure_time(model, val_loader, torch.device('cpu'))

        # Добавление результатов
        results.append({
            'Модель': model_name,
            'Точность Top-1 (%)': round(accuracy, 2),
            'Количество параметров (М)': round(num_params_m, 2),
            'FLOPS (G)': round(flops_count, 2),
            'Время обработки на GPU (мс)': round(gpu_time, 2) if gpu_time else 'N/A',
            'Время обработки на CPU (мс)': round(cpu_time, 2),
        })

# Вывод результатов в табличном виде
df = pd.DataFrame(results)
print("\nТаблица 8.1 – Сравнительная таблица производительности моделей на кастомном датасете:")
print(df)

# df.to_csv('model_comparison_custom_dataset.csv', index=False)


Тестируем модель: EfficientNet-B0

Тестируем модель: ResNet-50

Тестируем модель: MobileNetV2

Таблица 8.1 – Сравнительная таблица производительности моделей на кастомном датасете:
            Модель  Точность Top-1 (%)  Количество параметров (М)  FLOPS (G)  \
0  EfficientNet-B0                76.3                        5.3       0.39   
1        ResNet-50                76.1                       25.6       4.10   
2      MobileNetV2                71.8                        3.5       0.30   

   Время обработки на GPU (мс)  Время обработки на CPU (мс)  
0                         16.3                           75  
1                          9.6                          115  
2                          6.1                           55  
